# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
import pickle
import plotly
from plotly.graph_objs import Bar
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\black\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\black\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql('SELECT * FROM disaster_response', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.groupby('genre').count()['message']

genre
direct    10766
news      13054
social     2396
Name: message, dtype: int64

In [5]:
[df.original.notnull().sum(), df.original.isnull().sum()]

[10170, 16046]

In [9]:
lengths = df.message.str.split().str.len()
length_counts, length_division = np.histogram(lengths,range=(0, lengths.quantile(0.99)))
print(length_counts)

[ 834 5953 5290 5736 3596 2569 1184  418  245  137]


In [12]:
print(length_counts)
print(length_division)

[ 834 5953 5290 5736 3596 2569 1184  418  245  137]
[ 0.   6.6 13.2 19.8 26.4 33.  39.6 46.2 52.8 59.4 66. ]


In [13]:
df_categories = df.drop(["id", "message", "original", "genre"], axis=1)
df_categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df_categories.shape

(26216, 36)

In [14]:
df_categories[df_categories!=0].sum()

related                   20282.0
request                    4474.0
offer                       118.0
aid_related               10860.0
medical_help               2084.0
medical_products           1313.0
search_and_rescue           724.0
security                    471.0
military                    860.0
child_alone                   0.0
water                      1672.0
food                       2923.0
shelter                    2314.0
clothing                    405.0
money                       604.0
missing_people              298.0
refugees                    875.0
death                      1194.0
other_aid                  3446.0
infrastructure_related     1705.0
transport                  1201.0
buildings                  1333.0
electricity                 532.0
tools                       159.0
hospitals                   283.0
shops                       120.0
aid_centers                 309.0
other_infrastructure       1151.0
weather_related            7297.0
floods        

In [18]:
(df_categories[df_categories!=0].sum()/df_categories.shape[0]).round(decimals=3)

related                   0.774
request                   0.171
offer                     0.005
aid_related               0.414
medical_help              0.079
medical_products          0.050
search_and_rescue         0.028
security                  0.018
military                  0.033
child_alone               0.000
water                     0.064
food                      0.111
shelter                   0.088
clothing                  0.015
money                     0.023
missing_people            0.011
refugees                  0.033
death                     0.046
other_aid                 0.131
infrastructure_related    0.065
transport                 0.046
buildings                 0.051
electricity               0.020
tools                     0.006
hospitals                 0.011
shops                     0.005
aid_centers               0.012
other_infrastructure      0.044
weather_related           0.278
floods                    0.082
storm                     0.093
fire    

In [19]:
output_categories_percentage =  (df_categories[df_categories!=0].sum()/df_categories.shape[0]).round(decimals=3)
output_categories_percentage.index

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [20]:
output_categories_percentage.tolist()

[0.774,
 0.171,
 0.005,
 0.414,
 0.079,
 0.05,
 0.028,
 0.018,
 0.033,
 0.0,
 0.064,
 0.111,
 0.088,
 0.015,
 0.023,
 0.011,
 0.033,
 0.046,
 0.131,
 0.065,
 0.046,
 0.051,
 0.02,
 0.006,
 0.011,
 0.005,
 0.012,
 0.044,
 0.278,
 0.082,
 0.093,
 0.011,
 0.094,
 0.02,
 0.052,
 0.194]

In [ ]:
output_categories = output_categories_percentage.index.tolist()
category_frequency = output_categories_percentage.tolist()

In [3]:
X2 = df.filter(items=['id', 'message', 'original', 'genre'])
X2.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [4]:
X = df[['id', 'message', 'original', 'genre']]
X

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct
...,...,...,...,...
26211,30261,The training demonstrated how to enhance micro...,None,news
26212,30262,A suitable candidate has been selected and OCH...,None,news
26213,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news
26214,30264,"Some 2,000 women protesting against the conduc...",None,news


In [5]:
Y = df.drop(['id', 'message', 'original', 'genre', 'child_alone'], axis=1) #'child_alone' has no responses
Y['related'] = Y['related'].map(lambda x: 1 if x==2 else x)
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
categories = Y.columns.values
categories

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related',
       'transport', 'buildings', 'electricity', 'tools', 'hospitals',
       'shops', 'aid_centers', 'other_infrastructure', 'weather_related',
       'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'], dtype=object)

In [12]:
categories[0]

'related'

In [11]:
categories2 = Y.columns.tolist()
categories2

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [13]:
categories2[0]

'related'

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(token).lower().strip() for token in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [31]:
features_train, features_test, labels_train, labels_test = train_test_split(X, Y)
pipeline.fit(features_train['message'], labels_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000027E4EE14DC8>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [32]:
# Generate predictions
labels_pred = pipeline.predict(features_test['message'])

# Print out the full classification report
print(classification_report(labels_test, labels_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5047
               request       0.87      0.43      0.58      1124
                 offer       0.00      0.00      0.00        28
           aid_related       0.79      0.61      0.69      2717
          medical_help       0.56      0.03      0.06       532
      medical_products       0.95      0.06      0.11       344
     search_and_rescue       0.50      0.02      0.03       166
              security       0.00      0.00      0.00       119
              military       0.71      0.05      0.09       216
                 water       0.89      0.21      0.35       401
                  food       0.88      0.37      0.52       714
               shelter       0.88      0.22      0.36       585
              clothing       1.00      0.03      0.05       108
                 money       1.00      0.03      0.05       150
        missing_people       0.00      

C:\Users\black\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\black\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\black\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

### 6. Improve your model

Use grid search to find better parameters. 

In [41]:
parameters = {
    'vect__max_df': [0.8, 1.0],
    'clf__estimator__n_estimators': [10, 20],
    'clf__estimator__min_samples_split': [2, 5]    
}

# Initialize a gridsearchcv object that is parallelized
cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose=1)
cv.fit(features_train['message'], labels_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000027E4EE14DC8>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2, 5],
                         'clf__estimator__n_estimators': [10, 20],
                         'vect__max_df': [0.8, 1.0]},
             scoring='f1_micro', verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [42]:
# Generate predictions
labels_pred = cv.predict(features_test['message'])
# Print out the full classification report
print(classification_report(labels_test, labels_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5047
               request       0.82      0.46      0.59      1124
                 offer       0.00      0.00      0.00        28
           aid_related       0.76      0.62      0.68      2717
          medical_help       0.59      0.05      0.09       532
      medical_products       0.77      0.05      0.09       344
     search_and_rescue       0.62      0.03      0.06       166
              security       0.00      0.00      0.00       119
              military       0.61      0.05      0.09       216
                 water       0.86      0.24      0.37       401
                  food       0.83      0.48      0.61       714
               shelter       0.80      0.27      0.41       585
              clothing       0.88      0.06      0.12       108
                 money       0.67      0.01      0.03       150
        missing_people       1.00      

C:\Users\black\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\black\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\black\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [43]:
cv.best_params_

{'clf__estimator__min_samples_split': 5,
 'clf__estimator__n_estimators': 20,
 'vect__max_df': 0.8}

### 9. Export your model as a pickle file

In [53]:
m = pickle.dumps(clf) 

In [57]:
pickle.dumps(cv,open('finalized_model.sav', 'wb'))

In [58]:
print(m)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.